In [1]:
!pip install deap

In [2]:
from deap import base
from deap import creator
from deap import tools

import random
import numpy
import mlp_hyperparameters_test
import elitism

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)


In [0]:
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

BOUNDS_LOW =  [ 5,  -5, -10, -20, 0,     0,     0.0001, 0    ]
BOUNDS_HIGH = [15,  10,  10,  10, 2.999, 2.999, 2.0,    2.999]
NUM_OF_PARAMS = len(BOUNDS_HIGH)

POPULATION_SIZE = 20
P_CROSSOVER = 0.9 
P_MUTATION = 0.5  
MAX_GENERATIONS = 5
HALL_OF_FAME_SIZE = 3
CROWDING_FACTOR = 10.0 

In [4]:
def classificationAccuracy(individual):
    return test.getAccuracy(individual),

test = mlp_hyperparameters_test.MlpHyperparametersTest(RANDOM_SEED)
toolbox = base.Toolbox()
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

for i in range(NUM_OF_PARAMS):
    toolbox.register("attribute_" + str(i), random.uniform, BOUNDS_LOW[i], BOUNDS_HIGH[i])

attributes = ()
for i in range(NUM_OF_PARAMS):
    attributes = attributes + (toolbox.__getattribute__("attribute_" + str(i)),)

toolbox.register("individualCreator", tools.initCycle, creator.Individual, attributes, n=1)
toolbox.register("populationCreator", tools.initRepeat, list, toolbox.individualCreator)
toolbox.register("evaluate", classificationAccuracy)
toolbox.register("select", tools.selTournament, tournsize=2)
toolbox.register("mate", tools.cxSimulatedBinaryBounded, low=BOUNDS_LOW, up=BOUNDS_HIGH, eta=CROWDING_FACTOR)
toolbox.register("mutate", tools.mutPolynomialBounded, low=BOUNDS_LOW, up=BOUNDS_HIGH, eta=CROWDING_FACTOR, indpb=1.0/NUM_OF_PARAMS)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [5]:
population = toolbox.populationCreator(n=POPULATION_SIZE)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("max", numpy.max)
stats.register("avg", numpy.mean)
hof = tools.HallOfFame(HALL_OF_FAME_SIZE)
population, logbook = elitism.eaSimpleWithElitism(population,
                                                  toolbox,
                                                  cxpb=P_CROSSOVER,
                                                  mutpb=P_MUTATION,
                                                  ngen=MAX_GENERATIONS,
                                                  stats=stats,
                                                  halloffame=hof,
                                                  verbose=True)

gen	nevals	max 	avg  
0  	20    	0.94	0.448
1  	16    	0.94	0.633
2  	15    	0.94	0.737333
3  	16    	0.946667	0.841333
4  	15    	0.946667	0.929667
5  	17    	0.946667	0.912333


In [6]:
print("- Best solution is: \n",
      test.formatParams(hof.items[0]),
      "\n => accuracy = ",
      hof.items[0].fitness.values[0])

- Best solution is: 
 'hidden_layer_sizes'=(8, 3, 6)
 'activation'='tanh'
 'solver'='lbfgs'
 'alpha'=1.380111654273422
 'learning_rate'='invscaling' 
 => accuracy =  0.9466666666666667
